In [1]:
import warnings
import os

from sympy import Symbol
from numpy.testing import assert_allclose
import numpy as np
from pycalphad import Database, Model, calculate, equilibrium, EquilibriumError, ConditionError
from pycalphad.codegen.callables import build_callables
from pycalphad.core.solver import SolverBase, Solver
from pycalphad.core.utils import get_state_variables
import pycalphad.variables as v
from pycalphad.tests.datasets import *

warnings.simplefilter("always", UserWarning) # so we can test warnings


In [2]:
AL2O3_ND2O3_ZRO2_DBF = Database(AL2O3_ND2O3_ZRO2_TDB)
alfeo=Database("Al-Fe-O_Lindwall_etal.TDB")

/Users/sunhui/Desktop/new-charge/pycalphad/pycalphad/io/tdb.py:974: UserWarning: The type definition character `W` was defined in the following phases: ['BCC_B2'], but no corresponding TYPE_DEFINITION line was found in the TDB.
  warnings.warn(f"The type definition character `{typechar}` was defined in the following phases: "


In [3]:
def test_charge_balance_constraint():
    """Phases with charged species are correctly calculated and charged balanced in equilibrium"""
    comps = ['FE', 'AL', 'O', 'VA']
    #phases = list(AL2O3_ND2O3_ZRO2_DBF.phases.keys())
    phases = ['HALITE']
    conds = {v.P: 101325, v.N: 1, v.T: 500, v.X('FE'): 0.2, v.X('O'): 0.6}
    # Higher point density is required for convergence. Lower point densities
    # Can result in either no phases, or only FLUO phase (incorrect)
    res = equilibrium(alfeo, comps, phases, conds, calc_opts={'pdens':2000}, verbose=True)
    # Values below checked with Thermo-Calc
    print('a=',res.GM.values)
    print(res.Y.values.squeeze())
    #assert np.allclose(res.NP.values.squeeze()[:2], [0.30164254, 0.69835646])
    # site fractions of ND2O3_A
    Y_ND2O3_A = res.Y.values.squeeze()[0, :5]
    Y_PYRO = res.Y.values.squeeze()[1, :]
    SPEC_CHG_ND2O3_A = np.array([3, 4, -2, -2, 0])  # (ND+3,ZR+4):(O-2):(O-2,VA)
    SITE_RATIO_ND2O3_A = np.array([2, 2, 3, 1, 1])  # 2:3:1
    SPEC_CHG_PYRO = np.array([3, 4, 3, 4, -2, 0, -2, -2, 0])  # (ND+3,ZR+4):(ND+3,ZR+4):(O-2,VA):(O-2):(O-2,VA)
    SITE_RATIO_PYRO = np.array([2, 2, 2, 2, 6, 6, 1, 1, 1])  # 2:2:6:1:1
    CHG_ND2O3_A = np.dot(Y_ND2O3_A*SPEC_CHG_ND2O3_A, SITE_RATIO_ND2O3_A)
    CHG_PYRO = np.dot(Y_PYRO*SPEC_CHG_PYRO, SITE_RATIO_PYRO)
    print('CHARGE ND2O3_A', CHG_ND2O3_A)
    print('CHARGE PYRO', CHG_PYRO)
    #assert np.isclose(CHG_ND2O3_A, 0)
    #assert np.isclose(CHG_PYRO, 0)
    print(Y_ND2O3_A)
    assert np.allclose(Y_ND2O3_A, [9.79497936e-01, 2.05020639e-02, 1.00000000e+00, 2.05020639e-02, 9.79497936e-01], rtol=5e-4)
    assert np.allclose(Y_PYRO, [9.99970071e-01, 2.99288042e-05, 3.83395063e-02, 9.61660494e-01, 9.93381787e-01, 6.61821340e-03, 1.00000000e+00, 1.39970285e-03, 9.98600297e-01], rtol=5e-4)


In [4]:
a=test_charge_balance_constraint()

Components: AL AL+3 AL1O1 AL1O2 AL2 AL2FE1O4 AL2O AL2O1 AL2O2 AL2O3 ALO ALO2 ALO3/2 FE FE+2 FE+3 FE+4 FE1O1 FE1O2 FE2 FE2O3 FEO FEO3/2 O O-2 O2 O3 VA
Phases: HALITE 
[done]
properties <pycalphad.core.light_dataset.LightDataset object at 0x159786a00> {'HALITE': <pycalphad.core.phase_rec.PhaseRecord object at 0x7f96a92c2e40>} <pycalphad.core.light_dataset.LightDataset object at 0x1597e48e0> ['N', 'P', 'T', 'X_FE', 'X_O'] [N, P, T] True <pycalphad.core.solver.Solver object at 0x159a70a60>
ori_x [1.00000000e+00 1.01325000e+05 5.00000000e+02 4.31076957e-01
 3.84891874e-04 2.95587294e-01 2.72950857e-01 1.00000000e+00]
change_x [1.00000000e+00 1.01325000e+05 5.00000000e+02 4.31076957e-01
 3.84891874e-04 2.95587294e-01 2.72950857e-01 1.00000000e+00]
ori_x [1.00000000e+00 1.01325000e+05 5.00000000e+02 2.30427630e-01
 2.05228401e-03 3.82179901e-01 3.85340186e-01 1.00000000e+00]
change_x [1.00000000e+00 1.01325000e+05 5.00000000e+02 2.30427630e-01
 2.05228401e-03 3.82179901e-01 3.85340186e-01 1.0

len 8 7
charge_factor
compset [CompositionSet(HALITE, [0.24960318 0.1713745  0.57902232], NP=0.5920686054979991, GM=-275676.9308584439)
 CompositionSet(HALITE, [0.14270971 0.23796479 0.6193255 ], NP=0.09858096867008913, GM=-218126.60520943857)
 CompositionSet(HALITE, [0.12332077 0.24268833 0.6339909 ], NP=0.30935042583191175, GM=-204617.04519082946)]
site [1.00000000e+00 1.01325000e+05 5.00000000e+02 3.33333332e-01
 1.03891412e-09 3.33333334e-01 3.33333333e-01 1.00000000e+00]
len 8 7
charge_factor
compset [CompositionSet(HALITE, [0.24960318 0.1713745  0.57902232], NP=0.5920686054979991, GM=-275676.9308584439)
 CompositionSet(HALITE, [0.14270971 0.23796479 0.6193255 ], NP=0.09858096867008913, GM=-218126.60520943857)
 CompositionSet(HALITE, [0.12332077 0.24268833 0.6339909 ], NP=0.30935042583191175, GM=-204617.04519082946)]
site [1.00000000e+00 1.01325000e+05 5.00000000e+02 3.33333256e-01
 5.67628223e-08 3.33333373e-01 3.33333314e-01 1.00000000e+00]
len 8 7
charge_factor
Yes1
Yes2 2.0823

charge_factor
compset [CompositionSet(HALITE, [0.24960318 0.1713745  0.57902232], NP=0.5920686054979991, GM=-275676.9308584439)
 CompositionSet(HALITE, [0.14270971 0.23796479 0.6193255 ], NP=0.09858096867008913, GM=-218126.60520943857)
 CompositionSet(HALITE, [0.12332077 0.24268833 0.6339909 ], NP=0.30935042583191175, GM=-204617.04519082946)]
site [1.00000000e+00 1.01325000e+05 5.00000000e+02 2.24896938e-02
 9.58661463e-01 5.06933094e-03 1.37795124e-02 1.00000000e+00]
len 8 7
charge_factor
Yes1
Yes2 0.03127053020461279
Yes3
ori_x [1.00000000e+00 1.01325000e+05 5.00000000e+02 3.33333333e-01
 3.15926496e-11 3.33333333e-01 3.33333333e-01 1.00000000e+00]
change_x [1.00000000e+00 1.01325000e+05 5.00000000e+02 3.33333333e-01
 3.15926496e-11 3.33333333e-01 3.33333333e-01 1.00000000e+00]
ori_x [1.00000000e+00 1.01325000e+05 5.00000000e+02 2.72045703e-03
 9.94785906e-01 7.55605964e-04 1.73803150e-03 1.00000000e+00]
change_x [1.00000000e+00 1.01325000e+05 5.00000000e+02 2.72045703e-03
 9.9478590

Yes2 0.0005160029650413955
Yes3
ori_x [1.00000000e+00 1.01325000e+05 5.00000000e+02 3.33333333e-01
 3.86431290e-14 3.33333333e-01 3.33333333e-01 1.00000000e+00]
change_x [1.00000000e+00 1.01325000e+05 5.00000000e+02 3.33333333e-01
 3.86431290e-14 3.33333333e-01 3.33333333e-01 1.00000000e+00]
ori_x [1.00000000e+00 1.01325000e+05 5.00000000e+02 1.64742889e-03
 5.13083114e-01 3.22963829e-01 1.62305629e-01 1.00000000e+00]
change_x [1.00000000e+00 1.01325000e+05 5.00000000e+02 1.64742889e-03
 5.13083114e-01 3.22963829e-01 1.62305629e-01 1.00000000e+00]
ori_x [1.00000000e+00 1.01325000e+05 5.00000000e+02 6.26193285e-03
 9.90552188e-01 3.66081750e-05 3.14927052e-03 1.00000000e+00]
change_x [1.00000000e+00 1.01325000e+05 5.00000000e+02 6.26193285e-03
 9.90552188e-01 3.66081750e-05 3.14927052e-03 1.00000000e+00]
compset [CompositionSet(HALITE, [0.24960318 0.1713745  0.57902232], NP=0.5920686054979991, GM=-275676.9308584439)
 CompositionSet(HALITE, [0.14270971 0.23796479 0.6193255 ], NP=0.098580

 2.61217973e-13 3.33333333e-01 3.33333333e-01 1.00000000e+00]
len 8 7
charge_factor
compset [CompositionSet(HALITE, [0.24960318 0.1713745  0.57902232], NP=0.5920686054979991, GM=-275676.9308584439)
 CompositionSet(HALITE, [0.14270971 0.23796479 0.6193255 ], NP=0.09858096867008913, GM=-218126.60520943857)
 CompositionSet(HALITE, [0.12332077 0.24268833 0.6339909 ], NP=0.30935042583191175, GM=-204617.04519082946)]
site [1.00000000e+00 1.01325000e+05 5.00000000e+02 5.68542014e-03
 4.65603452e-01 3.50578945e-01 1.78132183e-01 1.00000000e+00]
len 8 7
charge_factor
compset [CompositionSet(HALITE, [0.24960318 0.1713745  0.57902232], NP=0.5920686054979991, GM=-275676.9308584439)
 CompositionSet(HALITE, [0.14270971 0.23796479 0.6193255 ], NP=0.09858096867008913, GM=-218126.60520943857)
 CompositionSet(HALITE, [0.12332077 0.24268833 0.6339909 ], NP=0.30935042583191175, GM=-204617.04519082946)]
site [1.00000000e+00 1.01325000e+05 5.00000000e+02 1.78290813e-02
 9.70636379e-01 1.74666589e-03 9.78787

...

 CompositionSet(HALITE, [0.12332077 0.24268833 0.6339909 ], NP=0.30935042583191175, GM=-204617.04519082946)]
site [1.00000000e+00 1.01325000e+05 5.00000000e+02 1.17267977e-03
 3.50707163e-01 4.31689211e-01 2.16430946e-01 1.00000000e+00]
len 8 7
charge_factor
compset [CompositionSet(HALITE, [0.24960318 0.1713745  0.57902232], NP=0.5920686054979991, GM=-275676.9308584439)
 CompositionSet(HALITE, [0.14270971 0.23796479 0.6193255 ], NP=0.09858096867008913, GM=-218126.60520943857)
 CompositionSet(HALITE, [0.12332077 0.24268833 0.6339909 ], NP=0.30935042583191175, GM=-204617.04519082946)]
site [1.00000000e+00 1.01325000e+05 5.00000000e+02 2.98750121e-03
 9.95032231e-01 3.24345094e-04 1.65592316e-03 1.00000000e+00]
len 8 7
charge_factor
Yes1
Yes2 0.004659350351134295
Yes3
ori_x [1.00000000e+00 1.01325000e+05 5.00000000e+02 3.33333333e-01
 1.21027712e-12 3.33333333e-01 3.33333334e-01 1.00000000e+00]
change_x [1.00000000e+00 1.01325000e+05 5.00000000e+02 3.33333333e-01
 1.21027712e-12 3.3333333

site [1.00000000e+00 1.01325000e+05 5.00000000e+02 2.08253285e-02
 9.63148618e-01 3.74225981e-03 1.22837941e-02 1.00000000e+00]
len 8 7
charge_factor
Yes2 0.02689641904493667
Yes3
ori_x [1.00000000e+00 1.01325000e+05 5.00000000e+02 3.33333333e-01
 1.17704503e-10 3.33333333e-01 3.33333334e-01 1.00000000e+00]
change_x [1.00000000e+00 1.01325000e+05 5.00000000e+02 3.33333333e-01
 1.17704503e-10 3.33333333e-01 3.33333334e-01 1.00000000e+00]
ori_x [1.00000000e+00 1.01325000e+05 5.00000000e+02 1.78429142e-01
 7.31069425e-01 8.57907834e-04 8.96435249e-02 1.00000000e+00]
change_x [1.00000000e+00 1.01325000e+05 5.00000000e+02 1.78429142e-01
 7.31069425e-01 8.57907834e-04 8.96435249e-02 1.00000000e+00]
ori_x [1.00000000e+00 1.01325000e+05 5.00000000e+02 6.21015335e-03
 9.90045037e-01 4.26488892e-04 3.31832113e-03 1.00000000e+00]
change_x [1.00000000e+00 1.01325000e+05 5.00000000e+02 6.21015335e-03
 9.90045037e-01 4.26488892e-04 3.31832113e-03 1.00000000e+00]
compset [CompositionSet(HALITE, [0.24

 CompositionSet(HALITE, [0.12332077 0.24268833 0.6339909 ], NP=0.30935042583191175, GM=-204617.04519082946)]
site [1.00000000e+00 1.01325000e+05 5.00000000e+02 1.67770723e-02
 9.72989568e-01 1.22988234e-03 9.00347731e-03 1.00000000e+00]
len 8 7
charge_factor
Yes2 0.3574884873632055
Yes3
ori_x [1.00000000e+00 1.01325000e+05 5.00000000e+02 3.33333333e-01
 7.65176624e-12 3.33333333e-01 3.33333334e-01 1.00000000e+00]
change_x [1.00000000e+00 1.01325000e+05 5.00000000e+02 3.33333333e-01
 7.65176624e-12 3.33333333e-01 3.33333334e-01 1.00000000e+00]
ori_x [1.00000000e+00 1.01325000e+05 5.00000000e+02 5.51985748e-01
 1.38125199e-01 2.25974522e-02 2.87291600e-01 1.00000000e+00]
change_x [1.00000000e+00 1.01325000e+05 5.00000000e+02 5.51985748e-01
 1.38125199e-01 2.25974522e-02 2.87291600e-01 1.00000000e+00]
ori_x [1.00000000e+00 1.01325000e+05 5.00000000e+02 2.35446428e-01
 6.15501081e-01 2.08861846e-02 1.28166306e-01 1.00000000e+00]
change_x [1.00000000e+00 1.01325000e+05 5.00000000e+02 2.3544

site [1.00000000e+00 1.01325000e+05 5.00000000e+02 1.26555535e-02
 3.22587366e-01 4.38952869e-01 2.25804211e-01 1.00000000e+00]
len 8 7
charge_factor
Yes1
Yes2 0.023151415619004134
Yes3
ori_x [1.00000000e+00 1.01325000e+05 5.00000000e+02 3.33333333e-01
 2.28467537e-11 3.33333333e-01 3.33333334e-01 1.00000000e+00]
change_x [1.00000000e+00 1.01325000e+05 5.00000000e+02 3.33333333e-01
 2.28467537e-11 3.33333333e-01 3.33333334e-01 1.00000000e+00]
ori_x [1.00000000e+00 1.01325000e+05 5.00000000e+02 1.27740465e-02
 9.67920074e-01 8.61257112e-03 1.06933088e-02 1.00000000e+00]
change_x [1.00000000e+00 1.01325000e+05 5.00000000e+02 1.27740465e-02
 9.67920074e-01 8.61257112e-03 1.06933088e-02 1.00000000e+00]
ori_x [1.00000000e+00 1.01325000e+05 5.00000000e+02 1.22996671e-02
 3.45738782e-01 4.23874478e-01 2.18087073e-01 1.00000000e+00]
change_x [1.00000000e+00 1.01325000e+05 5.00000000e+02 1.22996671e-02
 3.45738782e-01 4.23874478e-01 2.18087073e-01 1.00000000e+00]
compset [CompositionSet(HALITE,

KeyboardInterrupt: 

In [ ]:
a=test_charge_balance_constraint()

In [ ]:
model=Model(AL2O3_ND2O3_ZRO2_DBF,comps = ['ND', 'ZR', 'O', 'VA'],phase_name = 'ND2O3_A')

In [2]:
          ys=ys-chargefact*phr(jj)%invmat(nj,phr(jj)%idim)*&
               phr(jj)%curd%netcharge

SyntaxError: invalid syntax (2914376038.py, line 1)

In [5]:
import warnings
import os
import pytest
from sympy import Symbol
from numpy.testing import assert_allclose
import numpy as np
from pycalphad import Database, Model, calculate, equilibrium, EquilibriumError, ConditionError
from pycalphad.codegen.callables import build_callables, build_phase_records
from pycalphad.core.solver import SolverBase, Solver
from pycalphad.core.utils import get_state_variables, instantiate_models
import pycalphad.variables as v
from pycalphad.tests.datasets import *
ROSE_DBF = Database(ROSE_TDB)
ALFE_DBF = Database(ALFE_TDB)
ALNIFCC4SL_DBF = Database(ALNIFCC4SL_TDB)
ALCOCRNI_DBF = Database(ALCOCRNI_TDB)
ISSUE43_DBF = Database(ISSUE43_TDB)
TOUGH_CHEMPOT_DBF = Database(ALNI_TOUGH_CHEMPOT_TDB)
NI_AL_DUPIN_2001_DBF = Database(NI_AL_DUPIN_2001_TDB)
CUO_DBF = Database(CUO_TDB)
PBSN_DBF = Database(PBSN_TDB)
AL_PARAMETER_DBF = Database(AL_PARAMETER_TDB)
CUMG_PARAMETERS_DBF = Database(CUMG_PARAMETERS_TDB)


@pytest.mark.solver
def test_rose_nine():
    "Nine-component rose diagram point equilibrium calculation."
    my_phases_rose = ['TEST']
    comps = ['H', 'HE', 'LI', 'BE', 'B', 'C', 'N', 'O', 'F']
    conds = dict({v.T: 1000, v.P: 101325})
    for comp in comps[:-1]:
        conds[v.X(comp)] = 1.0/float(len(comps))
    eqx = equilibrium(ROSE_DBF, comps, my_phases_rose, conds, verbose=True)
    assert_allclose(eqx.GM.values.flat[0], -5.8351e3, atol=0.1)


@pytest.mark.solver
def test_eq_binary():
    "Binary phase diagram point equilibrium calculation with magnetism."
    my_phases = ['LIQUID', 'FCC_A1', 'HCP_A3', 'AL5FE2',
                 'AL2FE', 'AL13FE4', 'AL5FE4']
    comps = ['AL', 'FE', 'VA']
    conds = {v.T: 1400, v.P: 101325, v.X('AL'): 0.55}
    eqx = equilibrium(ALFE_DBF, comps, my_phases, conds, verbose=True)
    assert_allclose(eqx.GM.values.flat[0], -9.608807e4)
def test_eq_single_phase():
    "Equilibrium energy should be the same as for a single phase with no miscibility gaps."
    res = calculate(ALFE_DBF, ['AL', 'FE'], 'LIQUID', T=[1400, 2500], P=101325,
                    points={'LIQUID': [[0.1, 0.9], [0.2, 0.8], [0.3, 0.7],
                                       [0.7, 0.3], [0.8, 0.2]]})
    eq = equilibrium(ALFE_DBF, ['AL', 'FE'], 'LIQUID',
                     {v.T: [1400, 2500], v.P: 101325,
                      v.X('AL'): [0.1, 0.2, 0.3, 0.7, 0.8]}, verbose=True)
    assert_allclose(np.squeeze(eq.GM), np.squeeze(res.GM), atol=0.1)


/Users/sunhui/Desktop/new-charge/pycalphad/pycalphad/io/tdb.py:974: UserWarning: The type definition character `'` was defined in the following phases: ['FCC_A1'], but no corresponding TYPE_DEFINITION line was found in the TDB.
  warnings.warn(f"The type definition character `{typechar}` was defined in the following phases: "
/Users/sunhui/Desktop/new-charge/pycalphad/pycalphad/io/tdb.py:974: UserWarning: The type definition character `%` was defined in the following phases: ['FCC_A1'], but no corresponding TYPE_DEFINITION line was found in the TDB.
  warnings.warn(f"The type definition character `{typechar}` was defined in the following phases: "


In [8]:

test_rose_nine()

Components: B BE C F H HE LI N O
Phases: TEST 
[done]
properties <pycalphad.core.light_dataset.LightDataset object at 0x159fbf700> {'TEST': <pycalphad.core.phase_rec.PhaseRecord object at 0x7f96ab876f50>} <pycalphad.core.light_dataset.LightDataset object at 0x15a2dc910> ['N', 'P', 'T', 'X_B', 'X_BE', 'X_C', 'X_H', 'X_HE', 'X_LI', 'X_N', 'X_O'] [N, P, T] True <pycalphad.core.solver.Solver object at 0x15a04a7f0>
ori_x [1.00000000e+00 1.01325000e+05 1.00000000e+03 5.10204082e-01
 1.00000000e-14 1.00000000e-14 1.00000000e-14 1.00000000e-14
 1.00000000e-14 4.89795918e-01 1.00000000e-14 1.00000000e-14]
change_x [1.00000000e+00 1.01325000e+05 1.00000000e+03 5.10204082e-01
 1.00000000e-14 1.00000000e-14 1.00000000e-14 1.00000000e-14
 1.00000000e-14 4.89795918e-01 1.00000000e-14 1.00000000e-14]
ori_x [1.00000000e+00 1.01325000e+05 1.00000000e+03 1.00000000e-14
 5.10204082e-01 1.00000000e-14 1.00000000e-14 1.00000000e-14
 1.00000000e-14 1.00000000e-14 4.89795918e-01 1.00000000e-14]
change_x [1.0

  5.10204082e-01], NP=0.09831140740740762, GM=-5761.440646514652)                 ]
site [1.00000000e+00 1.01325000e+05 1.00000000e+03 3.51166461e-07
 3.51166461e-07 3.51166461e-07 4.99998771e-01 4.99998771e-01
 3.51167975e-07 3.51148493e-07 3.51187408e-07 3.51155139e-07]
len 12 9
compset [CompositionSet(TEST, [5.10204082e-01 1.00000000e-14 1.00000000e-14 1.00000000e-14
  1.00000000e-14 1.00000000e-14 4.89795918e-01 1.00000000e-14
  1.00000000e-14], NP=0.21777777777777765, GM=-5761.44064651465)
 CompositionSet(TEST, [1.00000000e-14 5.10204082e-01 1.00000000e-14 1.00000000e-14
  1.00000000e-14 1.00000000e-14 1.00000000e-14 4.89795918e-01
  1.00000000e-14], NP=0.2177777777777777, GM=-5761.44064651465)
 CompositionSet(TEST, [1.00000000e-14 1.00000000e-14 5.10204082e-01 1.00000000e-14
  1.00000000e-14 4.89795918e-01 1.00000000e-14 1.00000000e-14
  1.00000000e-14], NP=0.2177777777777777, GM=-5761.440646514652)
 CompositionSet(TEST, [1.00000000e-14 1.00000000e-14 1.00000000e-14 4.89795918e-0

len 12 9
compset [CompositionSet(TEST, [5.10204082e-01 1.00000000e-14 1.00000000e-14 1.00000000e-14
  1.00000000e-14 1.00000000e-14 4.89795918e-01 1.00000000e-14
  1.00000000e-14], NP=0.21777777777777765, GM=-5761.44064651465)
 CompositionSet(TEST, [1.00000000e-14 5.10204082e-01 1.00000000e-14 1.00000000e-14
  1.00000000e-14 1.00000000e-14 1.00000000e-14 4.89795918e-01
  1.00000000e-14], NP=0.2177777777777777, GM=-5761.44064651465)
 CompositionSet(TEST, [1.00000000e-14 1.00000000e-14 5.10204082e-01 1.00000000e-14
  1.00000000e-14 4.89795918e-01 1.00000000e-14 1.00000000e-14
  1.00000000e-14], NP=0.2177777777777777, GM=-5761.440646514652)
 CompositionSet(TEST, [1.00000000e-14 1.00000000e-14 1.00000000e-14 4.89795918e-01
  1.00000000e-14 5.10204082e-01 1.00000000e-14 1.00000000e-14
  1.00000000e-14], NP=0.008711111111111107, GM=-5761.440646514652)
 CompositionSet(TEST, [1.00000000e-14 1.00000000e-14 1.00000000e-14 4.89795918e-01
  5.10204082e-01 1.00000000e-14 1.00000000e-14 1.00000000e-

In [6]:
a=np.array([-1,2,-2,3])

In [9]:
x=a[2:]/2

In [18]:
a[1:].*a[1:]

SyntaxError: invalid syntax (3380728885.py, line 1)